In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Build the tokenizer

In [12]:
import os
import keras
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate

In [13]:
os.chdir('/content/drive/MyDrive/ICON_Task/Hindi')
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content/drive/.shortcut-targets-by-id/1nib99g_6T0IHcULEnV0SoCt0-hrkM0yM/Hindi': ['hindi_combined_hate_MACD.csv', 'train_hi_l1_modded.csv', 'glove', 'CNN_BiLSTM_ICON_Task-2_Hindi_Binary_Classification.h5', 'Hindi_Task2_output.csv', 'New_CNN+BiLSTM_Attempt-2_Hindi.ipynb', 'CNN_BiLSTM_ICON_Task-1_Hindi_Binary_Classification.h5', 'Hindi_Task1_output.csv', 'New_CNN+BiLSTM_Attempt-1_Hindi.ipynb', 'Fasttext', 'CNN_BiLSTM_ICON_Task-2_Hindi_Translated-Data_Binary_Classification.h5', 'Hindi_Task2_output_Translated-Model.csv', 'Translated_Dataset_New_CNN+BiLSTM_Attempt-2_Hindi.ipynb', 'XLM_Twitter_RoBERTa_Hindi.ipynb', 'CNN_BiLSTM_ICON_Task-3_Hindi_Binary_Classification.h5']


In [14]:
d1 = pd.read_csv('train_hi_l1_modded.csv')
d1 = d1.drop(['key', 'hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5'], axis=1)
d1['label'] = d1['label'].astype(int)
d1 = d1[['label', 'text']]
d1

,label,text
0,0,भारत में तीन किसान कानून बिल वापस लेने पर भक्त...
1,0,राजस्थान अजीबोगरीब: गांव की खुशहाली के लिए रा...
2,0,सलमान ने की राखी के विरोधी की वकालत तो 'आइटम ग...
3,0,!!हर शब्द अमॄतम!! पुरानी एक कहावत है... टूटी ...
4,1,"""PM मोदी जी "" की माँ के लिए अपशब्द कहने वाला द..."
...,...,...
6192,0,"😎 काम वाली बाई का फोन आया, पत्नि को। कामवाली ..."
6193,0,😎😎ग्रुप में चर्चा चल रही थी कि 21 दिन के लॉक ड...
6194,0,🚩 🔱▁▂▄▅▆▇█🦁█▇▆▅▄▂▁🔱 🔱⚜️👉 School में हमें स...
6195,0,"🚩 #लंका का जिहादी डॉक्टर,हजारों हिंदू महिलाओं ..."


In [15]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
d1['text'] = d1['text'].apply(lambda x: normalize_text(x))
d1

,label,text
0,0,भारत में तीन किसान कानून बिल वापस लेने पर भक्त...
1,0,राजस्थान अजीबोगरीब गांव की खुशहाली के लिए रा...
2,0,सलमान ने की राखी के विरोधी की वकालत तो आइटम ग...
3,0,हर शब्द अमॄतम पुरानी एक कहावत है टूटी ...
4,1,pm मोदी जी की माँ के लिए अपशब्द कहने वाला द...
...,...,...
6192,0,काम वाली बाई का फोन आया पत्नि को। कामवाली ...
6193,0,ग्रुप में चर्चा चल रही थी कि दिन के लॉक डाउन...
6194,0,school में हमें सती प्रथा घूंघट प्रथा ...
6195,0,लंका का जिहादी डॉक्टर हजारों हिंदू महिलाओं क...


In [17]:
X = list(d1['text'])
#y = np.asarray(d1[d1.columns[1:]])
y_data = d1['label']
y = y_data.values

In [18]:
max_len = 100
max_features = 10000
embed_size = 300
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, padding='post', maxlen=max_len)

# Load the Trained Model

In [6]:
os.chdir('/content/drive/MyDrive/ICON_Task/Hindi')
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content/drive/.shortcut-targets-by-id/1nib99g_6T0IHcULEnV0SoCt0-hrkM0yM/Hindi': ['hindi_combined_hate_MACD.csv', 'train_hi_l1_modded.csv', 'glove', 'CNN_BiLSTM_ICON_Task-2_Hindi_Binary_Classification.h5', 'Hindi_Task2_output.csv', 'New_CNN+BiLSTM_Attempt-2_Hindi.ipynb', 'CNN_BiLSTM_ICON_Task-1_Hindi_Binary_Classification.h5', 'Hindi_Task1_output.csv', 'New_CNN+BiLSTM_Attempt-1_Hindi.ipynb', 'Fasttext', 'CNN_BiLSTM_ICON_Task-2_Hindi_Translated-Data_Binary_Classification.h5', 'Hindi_Task2_output_Translated-Model.csv', 'Translated_Dataset_New_CNN+BiLSTM_Attempt-2_Hindi.ipynb', 'XLM_Twitter_RoBERTa_Hindi.ipynb']


In [7]:
model = keras.models.load_model('/content/drive/MyDrive/ICON_Task/Hindi/CNN_BiLSTM_ICON_Task-3_Hindi_Binary_Classification.h5')

# Testing on Test Data

In [8]:
import pandas as pd
vv= pd.read_csv('/content/test_label1.csv')
vv

,id,text,language,key
0,1,"""who's this tweet about"" bitch it's about you ...",english,question_1
1,2,#AlkaLamba should’ve used her slippers to slap...,english,question_1
2,3,#CoronaVillains #COVIDIDIOT #COVID2019 is a...,english,question_1
3,4,"(Acts 29:19994) LIKE the female ""governors"" of...",english,question_1
4,5,***Let us celebrate “secular” as a beautiful w...,english,question_1
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_1
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_1
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_1
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_1


In [9]:
vv = vv[vv['language'] == 'hindi']
vv

,id,text,language,key
1107,1108,#BandraStation #SharadPawar #Muradabad अगर अभ...,hindi,question_1
1108,1109,#ConspiracyAgainstIndia सुन लो रे देश के गद्द...,hindi,question_1
1109,1110,#MarathaReservation : महाराष्ट्र में जश्न का म...,hindi,question_1
1110,1111,#RheaChakraborty aap Mahesh bhatt se madad kiy...,hindi,question_1
1111,1112,"#SecularMaskOfd हमे पता नहि क्या कहते है ,इतना...",hindi,question_1
...,...,...,...,...
2618,2619,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,hindi,question_1
2619,2620,"🇮🇳जय हिन्द,🇮🇳जय भारत,🇮🇳वन्दे मातरम🇮🇳🇮🇳 बघेल तु...",hindi,question_1
2620,2621,🙏जय भीम 🙏 जो लोग हमें धमकियां देते है और संव...,hindi,question_1
2621,2622,🚩🕉🚩 *इंसान के पास 😇दिमाग होना चाहिये* *💀खोपड़...,hindi,question_1


In [10]:
vv1 = vv
vv1

,id,text,language,key
1107,1108,#BandraStation #SharadPawar #Muradabad अगर अभ...,hindi,question_1
1108,1109,#ConspiracyAgainstIndia सुन लो रे देश के गद्द...,hindi,question_1
1109,1110,#MarathaReservation : महाराष्ट्र में जश्न का म...,hindi,question_1
1110,1111,#RheaChakraborty aap Mahesh bhatt se madad kiy...,hindi,question_1
1111,1112,"#SecularMaskOfd हमे पता नहि क्या कहते है ,इतना...",hindi,question_1
...,...,...,...,...
2618,2619,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,hindi,question_1
2619,2620,"🇮🇳जय हिन्द,🇮🇳जय भारत,🇮🇳वन्दे मातरम🇮🇳🇮🇳 बघेल तु...",hindi,question_1
2620,2621,🙏जय भीम 🙏 जो लोग हमें धमकियां देते है और संव...,hindi,question_1
2621,2622,🚩🕉🚩 *इंसान के पास 😇दिमाग होना चाहिये* *💀खोपड़...,hindi,question_1


In [11]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
vv1['text'] = vv1['text'].apply(lambda x: normalize_text(x))
vv1

<ipython-input-11-01992e7e1f18>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vv1['text'] = vv1['text'].apply(lambda x: normalize_text(x))


,id,text,language,key
1107,1108,bandrastation sharadpawar muradabad अगर अभ...,hindi,question_1
1108,1109,conspiracyagainstindia सुन लो रे देश के गद्द...,hindi,question_1
1109,1110,marathareservation महाराष्ट्र में जश्न का म...,hindi,question_1
1110,1111,rheachakraborty aap mahesh bhatt se madad kiy...,hindi,question_1
1111,1112,secularmaskofd हमे पता नहि क्या कहते है इतना...,hindi,question_1
...,...,...,...,...
2618,2619,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,hindi,question_1
2619,2620,जय हिन्द जय भारत वन्दे मातरम बघेल तुझसे देश शर...,hindi,question_1
2620,2621,जय भीम जो लोग हमें धमकियां देते है और संविध...,hindi,question_1
2621,2622,इंसान के पास दिमाग होना चाहिये खोपड़ी तो ...,hindi,question_1


In [19]:
######## Textual Features for Embedding ###################
sequence = tokenizer.texts_to_sequences(vv1.text)
sequence = pad_sequences(sequence, padding='post', maxlen=max_len)

In [20]:
sequence = tokenizer.texts_to_sequences(vv1['text']) # same tokenizer which is used on train data.
sequence = pad_sequences(sequence,padding='post', maxlen = max_len)

In [21]:
import numpy as np

predictions = model.predict(sequence)
classes=np.argmax(predictions,axis=1)

48/48 [==============================] - 5s 78ms/step


In [22]:
predictions

array([[0.8275581 , 0.17244187],
       [0.505296  , 0.49470398],
       [0.811686  , 0.1883139 ],
       ...,
       [0.5789383 , 0.42106172],
       [0.72825074, 0.27174923],
       [0.85663444, 0.14336558]], dtype=float32)

In [23]:
pred = np.argmax(predictions, axis=1)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
ids = np.arange(1108, 2624)
# Create a DataFrame with 'id' and 'value' columns
result_df = pd.DataFrame({'id': ids, 'value': pred})
# result_df = pd.DataFrame({'value': predictions})
result_df

,id,value
0,1108,0
1,1109,0
2,1110,0
3,1111,0
4,1112,0
...,...,...
1511,2619,0
1512,2620,0
1513,2621,0
1514,2622,0


In [25]:
# Save the DataFrame to a CSV file
result_df.to_csv('/content/Hindi_Task3-l1_output.csv', index=False)

In [31]:
vv2= pd.read_csv('/content/test_label3.csv')
vv2

,id,text,language,key
0,1,"""who's this tweet about"" bitch it's about you ...",english,question_3
1,2,#AlkaLamba should’ve used her slippers to slap...,english,question_3
2,3,#CoronaVillains #COVIDIDIOT #COVID2019 is a...,english,question_3
3,4,"(Acts 29:19994) LIKE the female ""governors"" of...",english,question_3
4,5,***Let us celebrate “secular” as a beautiful w...,english,question_3
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_3
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_3
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_3
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_3


In [32]:
vv2 = vv2[vv2['language'] == 'hindi']
vv2

,id,text,language,key
1107,1108,#BandraStation #SharadPawar #Muradabad अगर अभ...,hindi,question_3
1108,1109,#ConspiracyAgainstIndia सुन लो रे देश के गद्द...,hindi,question_3
1109,1110,#MarathaReservation : महाराष्ट्र में जश्न का म...,hindi,question_3
1110,1111,#RheaChakraborty aap Mahesh bhatt se madad kiy...,hindi,question_3
1111,1112,"#SecularMaskOfd हमे पता नहि क्या कहते है ,इतना...",hindi,question_3
...,...,...,...,...
2618,2619,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,hindi,question_3
2619,2620,"🇮🇳जय हिन्द,🇮🇳जय भारत,🇮🇳वन्दे मातरम🇮🇳🇮🇳 बघेल तु...",hindi,question_3
2620,2621,🙏जय भीम 🙏 जो लोग हमें धमकियां देते है और संव...,hindi,question_3
2621,2622,🚩🕉🚩 *इंसान के पास 😇दिमाग होना चाहिये* *💀खोपड़...,hindi,question_3


In [33]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
vv2['text'] = vv2['text'].apply(lambda x: normalize_text(x))
vv2

<ipython-input-33-9ebfd64d3bb1>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vv2['text'] = vv2['text'].apply(lambda x: normalize_text(x))


,id,text,language,key
1107,1108,bandrastation sharadpawar muradabad अगर अभ...,hindi,question_3
1108,1109,conspiracyagainstindia सुन लो रे देश के गद्द...,hindi,question_3
1109,1110,marathareservation महाराष्ट्र में जश्न का म...,hindi,question_3
1110,1111,rheachakraborty aap mahesh bhatt se madad kiy...,hindi,question_3
1111,1112,secularmaskofd हमे पता नहि क्या कहते है इतना...,hindi,question_3
...,...,...,...,...
2618,2619,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,hindi,question_3
2619,2620,जय हिन्द जय भारत वन्दे मातरम बघेल तुझसे देश शर...,hindi,question_3
2620,2621,जय भीम जो लोग हमें धमकियां देते है और संविध...,hindi,question_3
2621,2622,इंसान के पास दिमाग होना चाहिये खोपड़ी तो ...,hindi,question_3


In [34]:
######## Textual Features for Embedding ###################
sequence = tokenizer.texts_to_sequences(vv2.text)
sequence = pad_sequences(sequence, padding='post', maxlen=max_len)

In [35]:
sequence = tokenizer.texts_to_sequences(vv2['text']) # same tokenizer which is used on train data.
sequence = pad_sequences(sequence,padding='post', maxlen = max_len)

In [36]:
import numpy as np

predictions = model.predict(sequence)
classes=np.argmax(predictions,axis=1)

48/48 [==============================] - 5s 111ms/step


In [37]:
predictions

array([[0.8275581 , 0.17244187],
       [0.505296  , 0.49470398],
       [0.811686  , 0.1883139 ],
       ...,
       [0.5789383 , 0.42106172],
       [0.72825074, 0.27174923],
       [0.85663444, 0.14336558]], dtype=float32)

In [38]:
pred = np.argmax(predictions, axis=1)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [39]:
ids = np.arange(1108, 2624)
# Create a DataFrame with 'id' and 'value' columns
result_df = pd.DataFrame({'id': ids, 'value': pred})
# result_df = pd.DataFrame({'value': predictions})
result_df

,id,value
0,1108,0
1,1109,0
2,1110,0
3,1111,0
4,1112,0
...,...,...
1511,2619,0
1512,2620,0
1513,2621,0
1514,2622,0


In [40]:
# Save the DataFrame to a CSV file
result_df.to_csv('/content/Hindi_Task3-l3_output.csv', index=False)